# PYSPARK PRACTICE

## Introduction

I'll be working with the following *dataset*: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data

We'll be working with high-level operations, so we need to create a SparkSession (a unified entrypoint for Spark that contains SparkContext, StreamingContext and SQLContext). If we wanted to work directly with RDDs, using a SparkContext would be more appropiate.

// TODO I'd like to test SparkContext at some point

In [1]:
from pyspark.sql import SparkSession

spark = (
   SparkSession.builder.appName('Spark Demo') 
  .master('local[*]') #local[*] - Run Spark locally with as many worker threads as logical cores on your machine. I could choose any number really
                    # YARN when deploying to a cluster with YARN
  .config("spark.sql.execution.arrow.maxRecordsPerBatch", "100")  # For testing purposes
  .getOrCreate()
)
spark

25/04/14 20:19:39 WARN Utils: Your hostname, alejandro resolves to a loopback address: 127.0.1.1; using 192.168.3.37 instead (on interface wlp2s0)
25/04/14 20:19:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 20:19:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


I won´t be using it here, but I should make an honorary mention to spark-submit, the script used to deploy jobs in a Spark cluster

./bin/spark-submit \
  --class <main-class> \
  --master <master-url> \
  --deploy-mode <deploy-mode> \
  --conf <key>=<value> \
  --py-files file1.py,file2.py,file3.zip,file4.egg \ # These will be passed to the worker nodes
  --archives conda_env.tag.gz \ # We could package the whole conda environment if we wanted to. Or a virtualEnv 
  ... # other options
  <application-jar> \
  [application-arguments]

Before we continue, some imports:

In [47]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

import numpy as np
import pandas as pd

## Explore the data - Introduction

Let's just load a csv file.

In [3]:
data_path = './spark_files_master/creditcard.csv'
df = spark.read.csv(data_path, header=True, inferSchema=True) # It is reading the file from my local file system, not the master or worker nodes
df.printSchema()

[Stage 1:=============================>                             (4 + 4) / 8]

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [4]:
df.show(1)

25/04/14 20:20:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------------+-------------------+----------------+----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+-------------------+------+-----+
|Time|              V1|                 V2|              V3|              V4|                V5|               V6|               V7|                V8|               V9|               V10|               V11|               V12|               V13|               V14|             V15|               V16|              V17|               V18|              V19|              V20|               V21|              V22|               V23|               V24|              V

Three important terms to remember here:
* *StructType*: A built-in DataType from org.apache.spark.sql.types that implements scala.collection.Seq<StructField>. Basically, it is a `Seq` of `StructField`. 
* *StructField*: The name, type and default `null` value of a row. You could see it as metadata of whatever you'll find in a row.
* *Row*: The values of the "column"

Let's do something a little bit more complicated. Let us show columns in groups of 4

In [5]:
def show_split(df, split=-1, n_samples=10):
    n_cols = len(df.columns)
    if split <= 0:
        split = n_cols
    i = 0
    j = i + split
    while i < n_cols:
        df.select(*df.columns[i:j]).show(n_samples) # That * operator will unpack the columsn from [c1, c2, c3] to c1, c2, c3
        i = j
        j = i + split
        
show_split(df, 4, 2)

+----+----------------+-------------------+----------------+
|Time|              V1|                 V2|              V3|
+----+----------------+-------------------+----------------+
| 0.0|-1.3598071336738|-0.0727811733098497|2.53634673796914|
| 0.0|1.19185711131486|   0.26615071205963|0.16648011335321|
+----+----------------+-------------------+----------------+
only showing top 2 rows

+-----------------+------------------+-------------------+-------------------+
|               V4|                V5|                 V6|                 V7|
+-----------------+------------------+-------------------+-------------------+
| 1.37815522427443|-0.338320769942518|  0.462387777762292|  0.239598554061257|
|0.448154078460911|0.0600176492822243|-0.0823608088155687|-0.0788029833323113|
+-----------------+------------------+-------------------+-------------------+
only showing top 2 rows

+------------------+------------------+------------------+------------------+
|                V8|            

We can gather quick information about certain columns by typing a simple line.

In [6]:
df.describe('V27', 'V28').show()

[Stage 11:>                                                         (0 + 8) / 8]

+-------+--------------------+--------------------+
|summary|                 V27|                 V28|
+-------+--------------------+--------------------+
|  count|              284807|              284807|
|   mean|-3.56859322007972...|-1.25938608605721...|
| stddev|  0.4036324949650301|  0.3300832641602508|
|    min|   -22.5656793207827|   -15.4300839055349|
|    max|    31.6121981061363|    33.8478078188831|
+-------+--------------------+--------------------+



We can be a little more specific with the values we want

In [7]:
# summary take statistics as params
df.select('V25', 'V26').summary('count', 'min', 'max', 'mean', '50%').show()

[Stage 14:>                                                         (0 + 8) / 8]

+-------+--------------------+--------------------+
|summary|                 V25|                 V26|
+-------+--------------------+--------------------+
|  count|              284807|              284807|
|    min|   -10.2953970749851|   -2.60455055280817|
|    max|    7.51958867870916|     3.5173456116238|
|   mean|7.153153300204557...|1.636403568872130...|
|    50%|  0.0164864026203845| -0.0521920661334584|
+-------+--------------------+--------------------+



As for today (04/2025), Python does not have access to the Dataset class; it can only use Dataframes. But, what is the difference between these two? And what makes them different to an RDD?
* At the core, an RDD is an immutable distributed collection of elements of your data, partitioned across nodes in your cluster that can be operated in parallel with a low-level API that offers transformations and actions.
  * They have no schema.
* Like an RDD, a DataFrame is an immutable distributed collection of data. Unlike an RDD, data is organized into named columns, allowing higher-level abstraction. We could see these structures as DataFrames[Row]
* Datasets are just strongly-typed DataFrames.

You cannot overlook the space efficiency and performance gains in using DataFrames and Dataset APIs for two reasons.
* First, because DataFrame and Dataset APIs are built on top of the Spark SQL engine, it uses Catalyst to generate an optimized logical and physical query plan. All relation type queries undergo the same code optimizer, providing the space and speed efficiency. Whereas the Dataset[T] typed API is optimized for data engineering tasks, the untyped Dataset[Row] (an alias of DataFrame) is even faster and suitable for interactive analysis.
* Second, since Spark as a compiler understands your Dataset type JVM object, it maps your type-specific JVM object to Tungsten's internal memory representation using Encoders. As a result, Tungsten Encoders can efficiently serialize/deserialize JVM objects as well as generate compact bytecode that can execute at superior speeds.
  * Tungsten is the codename for the umbrella project to make changes to Apache Spark’s execution engine that focuses on substantially improving the efficiency of memory and CPU for Spark applications

From: https://www.databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

# Explore the data in SQL-like fashion

We can use PySpark-like syntax. In the example below, I access columns in three different ways. All of them work, although the third one (`F.col`) does require an import.
What is the difference between them? https://stackoverflow.com/questions/55105363/pyspark-dataframe-column-reference-df-col-vs-dfcol-vs-f-colcol

In [8]:
df.where(df['Amount']>=100).select(df.Amount, F.col('Class')).show(10)

+-------+-----+
| Amount|Class|
+-------+-----+
| 149.62|    0|
| 378.66|    0|
|  123.5|    0|
|  121.5|    0|
| 231.71|    0|
|1402.95|    0|
| 120.96|    0|
| 169.05|    0|
| 243.66|    0|
| 135.51|    0|
+-------+-----+
only showing top 10 rows



We can also submit an SQL query directly.

`createOrReplaceTempView()` creates (or replaces if that view name already exists) a lazily evaluated "view". We need it to run queries this way. It does not persist to memory unless you cache the dataset that underpins the view. 

In [9]:
df.createOrReplaceTempView('df')
spark.sql('''
    SELECT Amount, Class FROM df
    WHERE AMount >= 100    
''').show(10)



+-------+-----+
| Amount|Class|
+-------+-----+
| 149.62|    0|
| 378.66|    0|
|  123.5|    0|
|  121.5|    0|
| 231.71|    0|
|1402.95|    0|
| 120.96|    0|
| 169.05|    0|
| 243.66|    0|
| 135.51|    0|
+-------+-----+
only showing top 10 rows



## Some more practice with more "complex" queries

Groups and filtering

In [10]:
(  
  df.where(df['Amount']>=100) 
  .groupBy('Class') 
  .agg(F.sum('amount').alias('sumAmount')) # This agg() defines how the values should merge, but it does not imply an aggregation of the 
                                            # values per se (Could be a count, an avg, min, max, variance, etc.).
  .where(F.col('sumAmount') > 300000)   # This second "where" acts like a HAVING (which in SQL acts as a where in aggregate functions anyway)
  .show(10)
)

+-----+--------------------+
|Class|           sumAmount|
+-----+--------------------+
|    0|1.9720083199999955E7|
+-----+--------------------+



Table unions (They work as a UNION ALL, with duplicates, unless stated otherwise)

In [15]:
df.union(df).distinct().count() # Maybe I need a better example for this

283726

Table Joins

In [ ]:
# Maybe I need a better example for this

## UDFs

What are UDFs? User-Defined Functions (aka UDF) is a feature of Spark SQL to define new Column-based functions that extend the vocabulary of Spark SQL's DSL for transforming Datasets.

In [37]:
def give_amount_classes(amount:int) -> str:
    if amount < 1000:
        return "Super low"
    if amount < 5000:
        return "Low"
    if amount < 10000:
        return "High"
    return "Super high"

# Let's create a UDF
giveAmountClasses = F.udf(give_amount_classes, T.StringType()) # Important!! Apparently, DataTypes are not singletons, so we have to "create" one by
                                                    # calling the constructor i.e. adding those "()"

And what if we do not want to register the UDF manually? There is a tag for it as well

In [43]:
@F.udf(T.StringType())
def myClassName(myClass:int) -> str:
    if myClass == 0:
        return "Class 0"
    return "Class 1"

In [45]:
df.select('Time', giveAmountClasses(df.Amount).alias('AmountString'), myClassName(df.Class).alias("myClassString")).show(10)

+----+------------+-------------+
|Time|AmountString|myClassString|
+----+------------+-------------+
| 0.0|   Super low|      Class 0|
| 0.0|   Super low|      Class 0|
| 1.0|   Super low|      Class 0|
| 1.0|   Super low|      Class 0|
| 2.0|   Super low|      Class 0|
| 2.0|   Super low|      Class 0|
| 4.0|   Super low|      Class 0|
| 7.0|   Super low|      Class 0|
| 7.0|   Super low|      Class 0|
| 9.0|   Super low|      Class 0|
+----+------------+-------------+
only showing top 10 rows



In [40]:
# Let's make sure we have other values of "AmountString" too.
# TODO Cache the above table so we don't have to recompute it here
df.select('Time', giveAmountClasses(df.Amount).alias('AmountString')).where(F.col("AmountString") == "Super high").show(10)

+--------+------------+
|    Time|AmountString|
+--------+------------+
| 42951.0|  Super high|
| 46253.0|  Super high|
| 48401.0|  Super high|
| 95286.0|  Super high|
|119713.0|  Super high|
|145283.0|  Super high|
|166198.0|  Super high|
|172273.0|  Super high|
+--------+------------+



What if we want to use this UDF with the SQL syntax? 

In [41]:
spark.udf.register('giveAmountClasses', give_amount_classes, T.StringType()) # We need to register the function with this one
                                                                            # The F.udf used above won't cut it 
spark.sql('''
    SELECT Time, giveAmountClasses(amount) AmountString
    FROM df
''').show(10)



+----+------------+
|Time|AmountString|
+----+------------+
| 0.0|   Super low|
| 0.0|   Super low|
| 1.0|   Super low|
| 1.0|   Super low|
| 2.0|   Super low|
| 2.0|   Super low|
| 4.0|   Super low|
| 7.0|   Super low|
| 7.0|   Super low|
| 9.0|   Super low|
+----+------------+
only showing top 10 rows



### Pandas UDF vs Python UDF


* In a Python UDF, when you pass column objects to your UDF, PySpark will unpack each value/row (convert the data between the python environment and the JVM, basically a serialization), perform the computation, and then return the value for each record.
* In a Scalar UDF, PySpark will serialize (through a library called PyArrow) each partitioned column into a pandas Series object. You then perform the operations on the Series object directly (avoiding the overhead of unpacking each row individually),returning a Series of the same dimension from your UDF.

From an end user perspective, they are the same functionally. Because pandas is optimized for rapid data manipulation, it is preferable to use a Series to Series UDF when you can instead of using a regular Python UDF, as it’ll be much faster.

Of course, this is only a problem for PySpark. If we were to use Spark and Scala/Java, that would be no problem.

From: https://gist.github.com/ThaiDat/81c3662801aa8410a65b94f3c993c377

In [48]:
@F.pandas_udf(T.StringType())
def getUserType(myClass: pd.Series) -> pd.Series:
    if myClass == 0:
        return "Class 0"
    return "Class 1"

# We can also promote the function to pandas as GetUserType = F.pandas_udf(get_user_type, T.StringType())
def give_amount_classes(amount:pd.Series) -> pd.Series:
    if amount < 1000:
        return "Super low"
    if amount < 5000:
        return "Low"
    if amount < 10000:
        return "High"
    return "Super high"

giveAmountClasses = F.udf(give_amount_classes, T.StringType())

We cannot use these UDFs with the SQL syntax.

In [50]:
df.select('Time', giveAmountClasses(df.Amount).alias('AmountString'), myClassName(df.Class).alias("myClassString")).show(10)

+----+------------+-------------+
|Time|AmountString|myClassString|
+----+------------+-------------+
| 0.0|   Super low|      Class 0|
| 0.0|   Super low|      Class 0|
| 1.0|   Super low|      Class 0|
| 1.0|   Super low|      Class 0|
| 2.0|   Super low|      Class 0|
| 2.0|   Super low|      Class 0|
| 4.0|   Super low|      Class 0|
| 7.0|   Super low|      Class 0|
| 7.0|   Super low|      Class 0|
| 9.0|   Super low|      Class 0|
+----+------------+-------------+
only showing top 10 rows



In [51]:
# Let's make sure we have other values of "AmountString" too.
# TODO Cache the above table so we don't have to recompute it here
df.select('Time', giveAmountClasses(df.Amount).alias('AmountString')).where(F.col("AmountString") == "Super high").show(10)

+--------+------------+
|    Time|AmountString|
+--------+------------+
| 42951.0|  Super high|
| 46253.0|  Super high|
| 48401.0|  Super high|
| 95286.0|  Super high|
|119713.0|  Super high|
|145283.0|  Super high|
|166198.0|  Super high|
|172273.0|  Super high|
+--------+------------+



TODO: Is there an UDF that could not be transformed into a Pandas' UDF?

Now, what if ,instead of passing a column to our UDF, we want to pass it two or three? In this case things are a little bit different.@pandas_udf(returnType=T.FloatType())
def tip_percent_of_fare(fares_breakdown: pd.DataFrame) -> pd.Series:
    print(fares_breakdown)
    result = (fares_breakdown['tip_amount']/fares_breakdown['total_amount'] * 100)
    print("!============================================!")
    return result

In [ ]:
@pandas_udf(returnType=T.FloatType())
def tip_percent_of_fare(fares_breakdown: pd.DataFrame) -> pd.Series:   # pd.DataFrame is the important difference here
    print(fares_breakdown)
    result = (fares_breakdown['tip_amount']/fares_breakdown['total_amount'] * 100)
    print("!============================================!")
    return result

These UDFs are processed in batches. What if there is an operation we do not want to repeat over an over every batch? What if we only want to do it once and use the result of said operation every batch? (loading a ML model, for example). In this case, instead of defining the input and output of our UDF as a pd.Series, we could define it as an iterator of batches of pd.Series.

In [ ]:
@pandas_udf("long")
def square_plus_y(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    y = y_bc.value  # initialize states
    try:
        for x in batch_iter:
            yield x * x + y
    finally:
        pass  # release resources here, if anytest_df.select(square_plus_y(col("id"))).show()

And what if we need to use this solution but use several columns as input? No problem either:

In [ ]:
def amount_mismatch(values: Iterator[Tuple[pd.Series, pd.Series, pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    # Heavy task
    # ...
    
    for oldOrig, newOrig, oldDest, newDest in values:
        yield abs(abs(newOrig - oldOrig) - abs(newDest - oldDest))
        
amountMismatch = F.pandas_udf(amount_mismatch, T.DoubleType())

In [ ]:
All the operations above return a series of the same lenght of the input. What if we want something different? We can also do that

In [ ]:
We can return only one value per batch

In [ ]:
@F.pandas_udf(T.DoubleType())
def GetStdDeviation(series: pd.Series) -> float:
    return series.std()

(
    df.groupBy('type')
    .agg(
        GetStdDeviation(df.amount).alias('var')
    )
    .orderBy('var', ascending=False)
    .show(10)
)



In [ ]:
Or a completely new struck!

In [ ]:
def normalize_by_type(data: pd.DataFrame) -> pd.DataFrame:
    result = data[['type', 'amount']].copy()
    maxVal = result['amount'].max()
    minVal = result['amount'].min()
    if maxVal == minVal:
        result['amountNorm'] = 0.5
    else:
        result['amountNorm'] = (result['amount'] - minVal) / (maxVal - minVal)
    return result

# We can use the SQL version of schema: 
# schema = 'type string, amount double, amountNorm double'
schema = T.StructType([
    T.StructField('type', T.StringType()),
    T.StructField('amount', T.DoubleType()),
    T.StructField('amountNorm', T.DoubleType())
])

(
    df.groupBy('type')
    .applyInPandas(normalize_by_type, schema)   # When using this function, we do not need to define the method as an UDF.
                                                # IMPORTANT! Apparently, this is now recommended in Spark 3 and the older method deprecated! 
                                                # But it can only be used when dealing with pd.DataFrame as inputs and outputs
    .show(10)
)

The function you need to use depends on the output:

| Input | Output | Function |
| --- | --- | --- |
| One row | One row | Pandas UDF |
| Many rows | One | `applyInPandas` |
| One row | Many rows | `mapInPandas()` |

In [1]:
## Windows